In [3]:
import pandas as pd
import matplotlib as plt
import seaborn as sns
import openpyxl
import statsmodels



In [ ]:
# get the old data sets back that I accidentally deleted

In [ ]:
stat_path

weight_path = 'UFC Fight Night Weights.xlsx'

In [4]:
# Step 1: Split fighters into separate columns
results_df[['Fighter_1', 'Fighter_2']] = results_df['BOUT'].str.split(' vs. ', expand=True)

fighter1_df = results_df.copy()
fighter1_df['FIGHTER'] = fighter1_df['Fighter_1']
fighter1_df['RESULT'] = fighter1_df['OUTCOME'].str[0].map({'W': 'Win', 'L': 'Loss'})
fighter1_df = fighter1_df.drop(columns=['Fighter_1','Fighter_2'])


fighter2_df = results_df.copy()
fighter2_df['FIGHTER'] = fighter2_df['Fighter_2']
fighter2_df['RESULT'] = fighter2_df['OUTCOME'].str[2].map({'W': 'Win', 'L': 'Loss'})
fighter2_df = fighter2_df.drop(columns=['Fighter_1','Fighter_2'])


# Note to self, clean this code so that I first drop fighter_1 and fighter_2, then combine the data sets together/

# Combine both into a single dataframe
results_df_clean = pd.concat([fighter1_df, fighter2_df])

results_df_clean['UFC_EVENT'] = results_df_clean['EVENT'].str.extract(r'(UFC \d+)', expand=False)





NameError: name 'results_df' is not defined

In [28]:
weight_df['UFC_EVENT'] = weight_df['EVENT'].str.extract(r'(UFC \d+)', expand=False)
weight_df

weight_df['FIGHTER'] = weight_df['FIGHTER'].str.strip().str.lower()
weight_df['UFC_EVENT'] = weight_df['UFC_EVENT'].str.strip().str.upper()

results_df_clean['FIGHTER'] = results_df_clean['FIGHTER'].str.strip().str.lower()
results_df_clean['UFC_EVENT'] = results_df_clean['UFC_EVENT'].str.strip().str.upper()

merged_weight = pd.merge(
    results_df_clean,
    weight_df,
    on=['FIGHTER', 'UFC_EVENT'],
    how='left', 
    suffixes=('_result', '_weight')
)
results_df_clean

# Drop rows with any NaN values
merged_weight_clean = merged_weight.dropna(subset=['WEIGH IN WEIGHT (lbs)'])


In [32]:
#df.groupby('FIGHTER')['SIG.STR. %'].mean()

# Log odds of winning based on how much weight you cut
win_weight_df = merged_weight_clean[['FIGHTER','RESULT','WEIGH IN WEIGHT (lbs)', 'FIGHT NIGHT WEIGHT (lbs)', 'WEIGHT INCREASE (lbs)']]

win_weight_df['RESULT'] = win_weight_df['RESULT'].map({'Win': 1, 'Loss': 0})
win_weight_df
#win_weight_df

/var/folders/rc/yw7v7vhj4l3_vjznh0cwm8wr0000gn/T/ipykernel_89384/2021029604.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  win_weight_df['RESULT'] = win_weight_df['RESULT'].map({'Win': 1, 'Loss': 0})


,FIGHTER,RESULT,WEIGH IN WEIGHT (lbs),FIGHT NIGHT WEIGHT (lbs),WEIGHT INCREASE (lbs)
17,nate landwehr,0.0,145.5,161.4,15.9
128,da'mon blackshear,1.0,134.0,152.4,18.4
146,josh emmett,0.0,146.0,166.8,20.8
156,martin buday,1.0,266.0,290.4,24.4
183,marvin vettori,0.0,185.5,211.0,25.5
...,...,...,...,...,...
13453,nina nunes,0.0,125.5,130.2,4.7
13957,germaine de randamie,0.0,136.0,148.0,12.0
14079,germaine de randamie,1.0,136.0,148.0,12.0
14156,darren elkins,0.0,145.5,167.4,21.9
